In [1]:
import pandas as pd
import re
import fire
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import os
from docx import Document
from tqdm.auto import tqdm
import pandas as pd
import ast
import sqlite3
import warnings
import time
import re

2024-12-01 15:51:04.830872: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 15:51:04.837923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733057464.847317  893290 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733057464.849900  893290 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 15:51:04.859782: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
df = pd.read_csv('После_LLM_buch_dataset.csv')

def split_and_expand_safe(df, columns):
    for col in columns:
        df[col] = df[col].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x
        )
    
    # Check lengths of lists in each row
    df['len_тема'] = df['Тема'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    df['len_описание'] = df['Описание'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
    # Remove rows with mismatched list lengths
    df = df[df['len_тема'] == df['len_описание']]
    
    # Remove the helper columns
    df = df.drop(columns=['len_тема', 'len_описание'])
    
    # Explode the specified columns
    df_expanded = df.explode(columns)
    return df_expanded

columns_to_process = ['Тема', 'Описание']
df = split_and_expand_safe(df, columns_to_process)
df = df.dropna()
df.shape, df['Тема'].nunique()

((17667, 7), 8309)

In [3]:
SYSTEM_PROMPT = """
Ты — эксперт в области бухгалтерии. Твоя задача — классифицировать информацию, связанную с бухгалтерией, налогами, отчетностью, документами юридических лиц, индивидуальными предпринимателями и другими смежными темами.
Контекст:
Дается текст, содержащий информацию, релевантную бухгалтерии, бизнесу или предпринимательской деятельности. Твоя задача — структурировать его, выделить ключевую тему и отнести ее к одной из указанных категорий.
Правила классификации:
1. Определи основную тему текста.
2. Отнеси ее к одной из следующих категорий:
   - Налоги, налогообложение и налоговое планирование
   - Бухгалтерский учет и финансовая отчетность
   - Финансовая отчетность и анализ
   - Внутренний контроль и аудит
   - Финансовое планирование, прогнозирование и анализ рисков
   - Учет заработной платы
   - Кадровое делопроизводство
   - Программное обеспечение для бухгалтерии и автоматизация
   - Взаимодействие с государственными органами
   - Правовое регулирование бухгалтерии, налогообложения и трудового законодательства
   - Финансовый анализ и оценка
3. Если подходящей категории нет, предложи новую, или верни "". Предложенные категории не изменяй и от себя ни чего не пиши, в скобках уточнение не добавляй.
4. Не указывай пересечения — выбери только одну категорию, наиболее соответствующую теме.

Результат:
Верни результат в формате только названия категории, к которой относится тема. 
"""

In [4]:
model_path = 'IlyaGusev/saiga_llama3_8b'  #название модели https://huggingface.co/
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Блок квантования
# # # Configure 8-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True, # Используется 8-битное квантован
    bnb_8bit_quant_type='nf4',  # 'nf4'
)

# # # # Configure 4-bit quantization
# quantization_config = BitsAndBytesConfig(
#     load_in_8bit=False,  # Не используется 8-битное квантование
#     load_in_4bit=True,   # Включаем 4-битное квантование
#     bnb_4bit_quant_type='fp4',  # 4-битное квантование с плавающей точкой
# )

# # Load model with quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},  # Использование bfloat16 в пайплайне
    device_map="auto",                             # Автоматическое распределение модели по устройствам
    pad_token_id=tokenizer.eos_token_id,
    truncation=True,
)

# Функция для инференса LLM
def interact(
    user_message,
    system_prompt=SYSTEM_PROMPT,
    temperature=0.1,
    max_length=512
):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt")
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
    
    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=128,
            temperature=temperature,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=pad_token_id
        )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Unused kwargs: ['bnb_8bit_quant_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

In [7]:
dict_category = {}
start_time = time.time()
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for topic in tqdm(df['Тема'].unique()):
        res = interact(topic.strip(), SYSTEM_PROMPT)
        dict_category[topic] = res

  0%|          | 0/8309 [00:00<?, ?it/s]

In [53]:
df['Категория'] = df['Тема'].replace(dict_category)
df['Категория'] = df['Категория'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df['Категория'] = df['Категория'].str.replace('"', '', regex=False)
df['Категория'] = df['Категория'].apply(
    lambda x: 'Бухгалтерский учет'
    if isinstance(x, str) and 'Бухгалтерский учет' in x
    else x
)
df['Категория'] = df['Категория'].apply(
    lambda x: 'Внутренний контроль и аудит'
    if isinstance(x, str) and 'Внутренний контроль' in x
    else x
)
df['Категория'] = df['Категория'].apply(
    lambda x: 'Кадровое делопроизводство'
    if isinstance(x, str) and 'кадровое' in x.lower()
    else x
)


df['Категория'] = df['Категория'].apply(
    lambda x: 'Налоги, налогообложение и налоговое планирование'
    if isinstance(x, str) and 'Налог' in x
    else x
)

df['Категория'] = df['Категория'].apply(
    lambda x: 'Взаимодействие с государственными органами'
    if isinstance(x, str) and 'Взаимодействие с государственными органами' in x
    else x
)

df['Категория'] = df['Категория'].apply(
    lambda x: 'Юридическое регулирование'
    if isinstance(x, str) and 'регулирование' in x.lower()
    else x
)

df['Категория'] = df['Категория'].apply(
    lambda x: 'Финансовое планирование и прогнозирование'
    if isinstance(x, str) and 'финансовое планирование' in x.lower()
    else x
)

df['Категория'] = df['Категория'].apply(
    lambda x: 'Правовые аспекты бухгалтерии и налогообложения'
    if isinstance(x, str) and 'правовые аспекты' in x.lower()
    else x
)

df['Категория'] = df['Категория'].apply(
    lambda x: 'Программное обеспечение для бухгалтерии и автоматизация'
    if isinstance(x, str) and 'программное обеспечение' in x.lower()
    else x
)


df['Категория'].nunique()

28

In [54]:
sorted(df['Категория'].unique())

['Бухгалтерские услуги',
 'Бухгалтерский учет',
 'Бухгалтерское образование и обучение',
 'Взаимодействие с банками и финансовыми учреждениями',
 'Взаимодействие с государственными органами',
 'Взаимодействие с клиентами',
 'Взаимодействие с налоговыми органами',
 'Внутренний контроль и аудит',
 'Кадровое делопроизводство',
 'Личностное развитие профессиональное',
 'Маркетинг, реклама, коммуникации',
 'Налоги, налогообложение и налоговое планирование',
 'Правовые аспекты бухгалтерии и налогообложения',
 'Предпринимательство',
 'Программное обеспечение для бухгалтерии и автоматизация',
 'Сотрудничество с государственными органами',
 'Управление кассой и денежными средствами',
 'Управление кредиторской задолженностью',
 'Управление проектами',
 'Учет дебиторской и кредиторской задолженности',
 'Учет для ИП',
 'Учет заработной платы',
 'Учет кадров и персонала',
 'Учет расходов',
 'Учет социальных взносов',
 'Финансовое планирование и прогнозирование',
 'Финансовый анализ и оценка',
 'Юри

In [55]:
df.dropna().to_csv('LLM_buch_dataset.csv', index=False)